In [1]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [3]:
from nltk import word_tokenize,sent_tokenize


In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Welcome\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents_short.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('Mychatbot_model.h5', hist)

print("model created")



29 documents
5 classes ['goodbye', 'greeting', 'identity', 'options', 'thanks']
48 unique lemmatized words ["'s", ',', 'about', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'for', 'good', 'goodbye', 'hai', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'human', 'is', 'later', 'me', "n't", 'next', 'nice', 'offered', 'provide', 'see', 'support', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'urself', 'what', 'who', 'you']
Training data created
Epoch 1/300
29/29 [==============================] - 0s 12ms/step - loss: 1.5832 - accuracy: 0.1379
Epoch 2/300
29/29 [==============================] - 0s 724us/step - loss: 1.5760 - accuracy: 0.2414
Epoch 3/300
29/29 [==============================] - 0s 620us/step - loss: 1.5908 - accuracy: 0.2759
Epoch 4/300
29/29 [==============================] - 0s 551us/step - loss: 1.5164 - accuracy: 0.4138
Epoch 5/300
29/29 [==============================] - 0s 521us/step - loss:

29/29 [==============================] - 0s 539us/step - loss: 0.0682 - accuracy: 0.9655
Epoch 77/300
29/29 [==============================] - 0s 539us/step - loss: 0.0209 - accuracy: 1.0000
Epoch 78/300
29/29 [==============================] - 0s 889us/step - loss: 0.0451 - accuracy: 1.0000
Epoch 79/300
29/29 [==============================] - 0s 539us/step - loss: 0.0470 - accuracy: 1.0000
Epoch 80/300
29/29 [==============================] - 0s 539us/step - loss: 0.0090 - accuracy: 1.0000
Epoch 81/300
29/29 [==============================] - 0s 539us/step - loss: 0.0569 - accuracy: 1.0000
Epoch 82/300
29/29 [==============================] - 0s 539us/step - loss: 0.0303 - accuracy: 1.0000
Epoch 83/300
29/29 [==============================] - 0s 539us/step - loss: 0.0330 - accuracy: 1.0000
Epoch 84/300
29/29 [==============================] - 0s 539us/step - loss: 0.0639 - accuracy: 1.0000
Epoch 85/300
29/29 [==============================] - 0s 539us/step - loss: 0.1066 - accuracy: 

29/29 [==============================] - 0s 724us/step - loss: 0.0867 - accuracy: 0.9655
Epoch 155/300
29/29 [==============================] - 0s 689us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 156/300
29/29 [==============================] - 0s 689us/step - loss: 0.0061 - accuracy: 1.0000
Epoch 157/300
29/29 [==============================] - 0s 620us/step - loss: 0.0154 - accuracy: 1.0000
Epoch 158/300
29/29 [==============================] - 0s 724us/step - loss: 0.0114 - accuracy: 1.0000
Epoch 159/300
29/29 [==============================] - 0s 655us/step - loss: 0.0044 - accuracy: 1.0000
Epoch 160/300
29/29 [==============================] - 0s 655us/step - loss: 0.0124 - accuracy: 1.0000
Epoch 161/300
29/29 [==============================] - 0s 620us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 162/300
29/29 [==============================] - ETA: 0s - loss: 0.0495 - accuracy: 1.00 - 0s 655us/step - loss: 0.0648 - accuracy: 0.9655
Epoch 163/300
29/29 [========================

29/29 [==============================] - 0s 586us/step - loss: 0.0232 - accuracy: 1.0000
Epoch 234/300
29/29 [==============================] - 0s 820us/step - loss: 0.0033 - accuracy: 1.0000
Epoch 235/300
29/29 [==============================] - 0s 792us/step - loss: 0.0100 - accuracy: 1.0000
Epoch 236/300
29/29 [==============================] - 0s 1ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 237/300
29/29 [==============================] - 0s 1ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 238/300
29/29 [==============================] - 0s 689us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 239/300
29/29 [==============================] - 0s 793us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 240/300
29/29 [==============================] - 0s 827us/step - loss: 0.0206 - accuracy: 1.0000
Epoch 241/300
29/29 [==============================] - 0s 758us/step - loss: 0.0081 - accuracy: 1.0000
Epoch 242/300
29/29 [==============================] - 0s 724us/step - loss: 0.0304 - accur